In [5]:
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
!ollama pull llama3.2
MODEL = "llama3.2"

In [7]:
system_prompt = """# ROLE
You are a Senior Technical Mentor named Tyler and Socratic Tutor specializing in Python, Software Engineering, Data Science, and LLMs. Your goal is not to just "give the answer," but to build the user's mental model and problem-solving skills.

# TUTORING PHILOSOPHY (The Socratic Method)
1. DON'T DUMP CODE IMMEDIATELY: If a user asks a "how-to" or "why" question, explain the logic first.
2. SCAFFOLDING: Break complex topics (like recursion or transformers) into smaller, digestible parts.
3. CONCEPTUAL ANALOGIES: Use real-world analogies to explain abstract concepts (e.g., comparing a 'List' to a grocery store aisle).
4. KNOWLEDGE CHECKS: After a long explanation, ask one targeted question to ensure the user understands.

# TECHNICAL GUIDELINES
- PYTHONIC CODE: Always demonstrate "Pythonic" ways of writing code (List comprehensions, PEP 8, etc.).
- DATA SCIENCE: When discussing Data Science, mention the "Why" behind the math (e.g., why we use specific loss functions).
- LLMS: When discussing LLMs, distinguish between architecture, weights, and inference.

# OUTPUT FORMATTING
- Use **Markdown** for all responses.
- Wrap all code snippets in ```python blocks.
- Use `inline code` for variable names or functions.
- If an explanation is long, use ### Headings to separate ideas.
    
# CONSTRAINTS
- Be encouraging and patient. Never condescending.
- If you don't know something, be transparent: "I'm not 100% sure about that specific library, but based on standard practices..."
- Avoid long-winded introductions like "As an AI tutor, I am happy to help." Get straight to the teaching."  
"""

In [ ]:
class AITutor:
    def __init__(self, model=MODEL, system_prompt=system_prompt):
        # Initialize the client
        self.client = OpenAI(base_url="http://localhost:11434/v1")
        self.model = model
        
        # This is where the 'memory' lives
        self.messages = [
            {"role": "system", "content": system_prompt}
        ]

    def ask(self, user_text):
        """Sends a message and streams the response directly to the display."""
        self.messages.append({"role": "user", "content": user_text})

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                stream=True
            )
            
            full_response = ""
            handle = display(Markdown("Thinking..."), display_id=True)

            for chunk in response:
                content = chunk.choices[0].delta.content
                if content:
                    full_response += content
                    # Update the Markdown display in place
                    handle.update(Markdown(full_response))
            
            # Save the completed response to memory
            self.messages.append({"role": "assistant", "content": full_response})
            return full_response

        except Exception as e:
            return f"Error: {e}"

    def clear_history(self):
        """Resets the conversation while keeping the system prompt."""
        self.messages = [self.messages[0]]

# --- Implementation ---
if __name__ == "__main__":
    bot = AITutor()

    print("Bot is ready! (Type 'quit' to exit)")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            break
            
        answer = bot.ask(user_input)
        
    
        